# 快速入门 MSG SEVIRI
在本教程中，我们将使用未压缩的 EUMETSAT HRIT 格式的 Meteosat 第二代（MSG）数据，使用 satpy 读取，使用 pyresample 进行重采样并进行一些处理。请确保已安装所需的 Python 包，例如：
`pip install satpy`
用于解压缩HRIT数据的软件称为公共小波变换解压缩库软件，可以从EUMETSAT [在他们的软件页面](https://www.eumetsat.int/website/home/Data/DataDelivery/SupportSoftwareandTools/index.html?lang=EN)获取。压缩后的HRIT数据文件可以通过文件名以`C_`结尾轻松识别，而未压缩的数据文件则以`__`（两个下划线）结尾。

## 加载数据和生成我们的第一个图像
使用 satpy 实时加载数据是在生成合成图像时进行的。因此，首先我们创建一个场景实例，将 `base_dir` 目录指向包含未压缩 HRIT 数据文件的位置。

In [4]:
from satpy.scene import Scene
from satpy.resample import get_area_def
from satpy import find_files_and_readers
from datetime import datetime
import glob
import warnings

# Date format in YYYYMMDDhhmm
fnames = glob.glob('/path/to/data/H*202009060000*__')
scn = Scene(reader='seviri_l1b_hrit', filenames=fnames)

然后我们定义一个组合来加载并显示它。您可能会看到由于弃用的 pyproj 功能而产生的警告，以及当 numpy/dask 处理 NaN 值时。NaN 用于标记无效像素和空间像素（例如您在全盘图像中会看到的情况）。你可以使用 `warnings.filterwarnings('ignore')` 来忽略这些警告。

In [5]:
composite = 'natural_color'
scn.load([composite])
scn.show(composite)

<img src='natural_hrit.png'>

正如您所见，地球的显示是北下南上：请记住，这些数据来自hrit文件，是未投影的原始数据，因此像素是按扫描顺序显示的。
要获取可选的复合材料列表：

In [4]:
scn.available_composite_ids()

[DatasetID(name='airmass', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='airmass_corr', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='ash', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='cloudtop', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='convection', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='day_microphysics', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='dust', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='fog', wavelength=None, resolution=None, polarization=None, calibration=None, modifiers=None),
 DatasetID(name='green_snow', wavelength=None, resolution=None, polarizat

## 加载频道数据并处理它
为了加载可用的通道数据，步骤与上述相同，只是需要传递通道名称或其波长（以及功能齐全的 `DatasetID`）。例如：

In [5]:
scn.load(['VIS006', 0.8])
print(scn)

<xarray.DataArray 'astype-7a343ed713478b648e6dadb526d4c114' (y: 3712, x: 3712)>
dask.array<concatenate, shape=(3712, 3712), dtype=float64, chunksize=(53, 3712)>
Coordinates:
  * x        (x) float64 -5.569e+06 -5.569e+06 -5.569e+06 -5.569e+06 ...
  * y        (y) float64 5.566e+06 5.566e+06 5.566e+06 5.566e+06 5.566e+06 ...
Attributes:
    units:                %
    wavelength:           (0.74, 0.81, 0.88)
    standard_name:        toa_bidirectional_reflectance
    platform_name:        Meteosat-10
    sensor:               seviri
    satellite_longitude:  0.0
    satellite_latitude:   0.0
    satellite_altitude:   35785831.0
    start_time:           2017-01-19 09:30:10.553000
    end_time:             2017-01-19 09:42:41.403000
    area:                 Area ID: some_area_name\nDescription: On-the-fly ar...
    name:                 VIS008
    resolution:           3000.40316582
    calibration:          reflectance
    polarization:         None
    modifiers:            ()
    anc

处理数据相当直接，因为场景中的数据集实际上是以 xarray Python 包中的 DataArray 形式存在的。

In [6]:
from satpy.dataset import combine_metadata
ndvi = (scn[0.8] - scn[0.6]) / (scn[0.8] + scn[0.6])
ndvi.attrs = combine_metadata(scn[0.8], scn[0.6])
scn['ndvi'] = ndvi
scn.show('ndvi')

/home/a001673/usr/src/dask/dask/local.py:271: RuntimeWarning: invalid value encountered in divide
  return func(*args2)


<img src="ndvi_hrit.png">

## 数据重采样
直到现在，我们一直直接使用卫星提供的通道，即卫星投影。因此生成合成图像会生成卫星投影的视图，即卫星所看到的视角。
然而，大多数情况下，我们希望将数据投影到特定区域，以便在RGB合成图像中仅显示感兴趣的区域。
以下是我们的做法：

In [7]:
local_scn = scn.resample("eurol")
print(local_scn)

<xarray.DataArray 'array-6cc44ebc64d08c71c8e71a0c2b12bf7a' (y: 2048, x: 2560)>
dask.array<transpose, shape=(2048, 2560), dtype=float64, chunksize=(1000, 1000)>
Coordinates:
  * y        (y) float64 -1.502e+06 -1.504e+06 -1.508e+06 -1.510e+06 ...
  * x        (x) float64 -3.778e+06 -3.776e+06 -3.772e+06 -3.77e+06 ...
Attributes:
    units:                %
    wavelength:           (0.74, 0.81, 0.88)
    standard_name:        toa_bidirectional_reflectance
    platform_name:        Meteosat-10
    sensor:               seviri
    satellite_longitude:  0.0
    satellite_latitude:   0.0
    satellite_altitude:   35785831.0
    start_time:           2017-01-19 09:30:10.553000
    end_time:             2017-01-19 09:42:41.403000
    area:                 Area ID: eurol\nDescription: Euro 3.0km area - Euro...
    name:                 VIS008
    resolution:           3000.40316582
    calibration:          reflectance
    polarization:         None
    modifiers:            ()
    ancillary_v

In [8]:
local_scn.show('ndvi')

<img src="ndvi_hrit_resampled.png">

In [10]:
local_scn.show('natural')

<img src="natural_hrit_resampled.png">